In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import click

In [2]:
import build_database

In [159]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [3]:
DATABASE = "processed_data/locator_database.csv"

In [294]:
build_database.main()

Processing housing data sources...
Flagging problem landlords...
Computing transit access...
Building the database...
Saving the database...
Finished


In [23]:
db = pd.read_csv(DATABASE, index_col="index")

In [35]:
db.head()

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,...,2016_evict_rate,2016_evict_filing_rate,2011-2015_rent_perc_change,2015-2019_rent_perc_change,potential_bad_landlord,bad_landlord_address,num_stops_quart_mi,num_stops_half_mi,num_stops_3quart_mi,num_stops_1_mi
index,,,,,,,,,,,,,,,,,,,,,
1288108,"1718 W 66th St 1, Chicago, IL 60636",800,Apt,1.0,2.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.774000,...,1.54,4.62,0.002481,0.113036,False,NaN,0,2,2,2
4012748,"6130 S Eberhart Ave 1, Chicago, IL 60637",1200,Apt,2.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.783000,...,2.81,8.15,-0.046617,0.026814,False,NaN,2,4,4,8
4017021,"4827 S Seeley Ave , Chicago, IL 60609",600,Apt,1.0,1.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.805800,...,0.42,2.09,0.000000,0.083398,False,NaN,0,2,2,2
4036551,"828 E 88th Pl 2, Chicago, IL 60619",875,Apt,1.0,3.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.734190,...,2.26,7.89,0.014996,0.097201,False,NaN,0,0,0,0
4036578,"4620 S Evans Ave 1, Chicago, IL 60653",1150,Apt,1.0,3.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.810646,...,2.73,5.45,0.002683,0.030100,False,NaN,0,0,4,6


Address (str): full address. e.g. "1718 W 66th St 1, Chicago, IL 60636"
Monthly Rent (tuple of int): rent within (<min of range>, <max of range>)
Property Type (list of str): a list of the following options: 
                    "4-Plex", "Apt", "Duplex", "House", "Townhouse", "TriPlex"
Bath (tuple of float): number of bathrooms within (<min of range>, <max of range>)
Bed (tuple of float): number of bedrooms within (<min of range>, <max of range>)
Requires Voucher (bool): whether the unit requires HCV
Available Now (bool): whether the unit is available now
Neighborhood (str): a zillow neighborhood
L-Stop within _ Mile (float): a float of the following options: 
                              0.25, 0.5, 0.75, 1


In [163]:
DIST_TO_COL = {0.25: "num_stops_quart_mi",
                0.5: "num_stops_half_mi",
                0.75: "num_stops_3quart_mi",
                1: "num_stops_1_mi"}

In [236]:
c = {
    "Address": None,
    "Monthly Rent": (800, 1000),
    "Property Type": ["Apt"],
    "Bath": (1, 2),
    "Bed": (1, 1),
    "Available Now": True,
    "Neighborhood": None,
    "Has L-Stop within _ Mile": 0.25}

In [149]:
c

{'Address': None,
 'Monthly Rent': (800, 1000),
 'Property Type': ['Apt', 'Duplex'],
 'Bath': (1, 2),
 'Bed': None,
 'Available Now': True,
 'Neighborhood': 'Woodlawn',
 'Distance to nearest L-Stop': None}

In [166]:
rv = db

In [151]:
if c["Address"]:
    f1 = rv.Address == c["Address"]
    rv = rv[f1]

In [152]:
if c["Monthly Rent"]:
    lb_rent, ub_rent = c["Monthly Rent"]
    f2 = (rv["Monthly Rent"] >= lb_rent) & (rv["Monthly Rent"] <= ub_rent)
    rv = rv[f2]

In [153]:
if c["Property Type"]:
    f3 = rv["Property Type"].isin(c["Property Type"])
    rv = rv[f3]

In [154]:
if c["Bath"]:
    lb_bath, ub_bath = c["Bath"]
    f4 = (rv.Bath >= lb_bath) & (rv.Bath <= ub_bath)
    rv = rv[f4]

In [155]:
if c["Bed"]:
    lb_bed, ub_bed = c["Bath"]
    f5 = (rv.Bed >= lb_bed) & (rv.Bed <= ub_bed)
    rv = rv[f5]

In [156]:
if c["Available Now"]:
    f6 = rv.Availability == "Available Now"
    rv = rv[f6]

In [157]:
if c["Neighborhood"]:
    f7 = rv.Neighborhood == c["Neighborhood"]
    rv = rv[f7]

In [168]:
if c["Has L-Stop within _ Mile"]:
    d = c["Has L-Stop within _ Mile"]
    f8 = rv[DIST_TO_COL[d]] > 0
    rv = rv[f8]

In [171]:
rv

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long,Coordinates,GEOID,State,County,City,Neighborhood,RegionID,2016_evict_rate,2016_evict_filing_rate,2011-2015_rent_perc_change,2015-2019_rent_perc_change,potential_bad_landlord,bad_landlord_address,num_stops_quart_mi,num_stops_half_mi,num_stops_3quart_mi,num_stops_1_mi
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4012748,"6130 S Eberhart Ave 1, Chicago, IL 60637",1200,Apt,2.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.783000,-87.613600,POINT (-87.61360000000001 41.783),170314206001,IL,Cook,Chicago,West Woodlawn,403353,2.81,8.15,-0.046617,0.026814,False,NaN,2,4,4,8
4062710,"211 N Laporte Ave 2, Chicago, IL 60644",950,Duplex,1.0,2.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.884900,-87.748800,POINT (-87.7488 41.8849),170312518005,IL,Cook,Chicago,South Austin,403120,0.99,4.28,-0.003479,0.053771,False,NaN,2,4,4,8
4062846,"6141 S Eberhart Ave 1, Chicago, IL 60637",1350,Apt,2.0,6.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.782700,-87.612900,POINT (-87.6129 41.7827),170314206002,IL,Cook,Chicago,West Woodlawn,403353,1.42,6.86,-0.046617,0.026814,False,NaN,2,4,4,8
4064659,"6110 S Vernon Ave G, Chicago, IL 60637",900,Apt,1.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.783500,-87.614700,POINT (-87.6147 41.7835),170314206001,IL,Cook,Chicago,West Woodlawn,403353,2.81,8.15,-0.046617,0.026814,False,NaN,2,2,4,8
4076796,"6110 S Vernon Ave , Chicago, IL 60637",1250,Apt,1.0,4.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.783500,-87.614700,POINT (-87.6147 41.7835),170314206001,IL,Cook,Chicago,West Woodlawn,403353,2.81,8.15,-0.046617,0.026814,False,NaN,2,2,4,8
4077614,"3334 W Walnut St , Chicago, IL 60624",1150,Apt,2.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.885600,-87.710100,POINT (-87.7101 41.8856),170318368002,IL,Cook,Chicago,East Garfield Park,269577,2.85,6.58,-0.012162,0.034200,False,NaN,2,4,6,10
4079071,"3334 W Walnut St , Chicago, IL 60624",1200,Apt,1.5,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.885600,-87.710100,POINT (-87.7101 41.8856),170318368002,IL,Cook,Chicago,East Garfield Park,269577,2.85,6.58,-0.012162,0.034200,False,NaN,2,4,6,10
4205319,"3509 S Western Blvd 2, Chicago, IL 60609",1000,Apt,1.0,3.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.829962,-87.684023,POINT (-87.684023 41.829962),170315905001,IL,Cook,Chicago,McKinley Park,139408,0.28,1.71,-0.005373,0.114787,False,NaN,2,2,2,2
4274287,"646 E 62nd St 2, Chicago, IL 60637",997,Apt,1.0,2.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.782300,-87.609000,POINT (-87.60899999999999 41.7823),170314205001,IL,Cook,Chicago,West Woodlawn,403353,0.70,2.63,-0.046617,0.026814,False,NaN,2,4,4,4


In [172]:
import user

In [216]:
user.search(c, db, user.OUTPUT1)

2 search results saved in output/sample_output1.csv


In [181]:
ptypes = ["4-Plex", "Apt", "Duplex", "House", "Townhouse", "TriPlex"]

In [183]:
print(str(ptypes))

['4-Plex', 'Apt', 'Duplex', 'House', 'Townhouse', 'TriPlex']


In [203]:
dist_to_col = {0.25: "num_stops_quart_mi",
                0.5: "num_stops_half_mi",
                0.75: "num_stops_3quart_mi",
                1: "num_stops_1_mi"}

In [217]:
pd.read_csv(user.OUTPUT1)

,index,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long,Coordinates,GEOID,State,County,City,Neighborhood,RegionID,2016_evict_rate,2016_evict_filing_rate,2011-2015_rent_perc_change,2015-2019_rent_perc_change,potential_bad_landlord,bad_landlord_address,num_stops_quart_mi,num_stops_half_mi,num_stops_3quart_mi,num_stops_1_mi
0,4530054,"5219 S Calumet Ave 1, Chicago, IL 60615",900,Apt,1.0,1.0,Yes,Available Now,(708) 473-5464,http://chicagoha.gosection8.com/Section-8-hous...,41.7997,-87.6172,POINT (-87.6172 41.7997),170318361003,IL,Cook,Chicago,Washington Park,275927,4.60,10.34,-0.033813,0.038719,False,NaN,2,4,6,8
1,4771491,"3900 W Van Buren St , Chicago, IL 60624",950,Apt,1.0,1.0,Yes,Available Now,(312) 918-5955,http://chicagoha.gosection8.com/Section-8-hous...,41.8755,-87.7233,POINT (-87.72329999999999 41.8755),170312608002,IL,Cook,Chicago,East Garfield Park,269577,2.59,7.49,-0.012162,0.034200,False,NaN,2,2,6,8


In [245]:
cr = user.Criteria()

In [246]:
cr

Address: None
Monthly Rent: None
Property Type: None
Bath: None
Bed: None
Available Now: None
Neighborhood: None
Has L-Stop within _ Mile: None

In [301]:
cr.add_criteria(c)

In [293]:
cr.clear_criteria()

All search fields are now cleared


In [332]:
cr.update_criteria({'Neighborhood': ["Love"]})

In [334]:
user.search(cr, user.DB, user.SAMPLE_OUTPUT)

ValueError: Wrong input value for Neighborhood
Input a list containing the following: dict_keys(['Albany Park', 'Andersonville', 'Arcadia Terrace', 'Archer Heights', 'Ashburn', 'Avalon Park', 'Avondale', 'Back of the Yards', 'Belmont Central', 'Belmont Gardens', 'Belmont Heights', 'Belmont Terrace', 'Beverly', 'Beverly Woods', 'Big Oaks', 'Bowmanville', 'Brainerd', 'Bridgeport', 'Brighton Park', 'Bronzeville', 'Bucktown', 'Budlong Woods', 'Burnside', 'Cabrini Green', 'Calumet Heights', 'Canaryville', 'Chatham', 'Chicago Lawn', 'Chinatown', 'Chrysler Village', 'Clearing', 'Cottage Grove Heights', 'Cragin', 'Dearborn Park', 'Dunning', 'East Beverly', 'East Chatham', 'East Garfield Park', 'East Hyde Park', 'East Side', 'East Ukrainian Village', 'Edgebrook', 'Edgewater', 'Edgewater Glen', 'Edison Park', 'Englewood', 'Fernwood', 'Ford City', 'Forest Glen', 'Fulton River District', 'Gage Park', 'Galewood', 'Garfield Ridge', 'Gold Coast', 'Goose Island', 'Graceland West', 'Grand Crossing', 'G

In [327]:
cr.add_criteria({"Available Now": False})

In [319]:
d1 = db[(db["Monthly Rent"] >= 800)  & (db["Monthly Rent"] <= 1000)]

In [326]:
d1[d1["Neighborhood"] == "Woodlawn"]

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long,Coordinates,GEOID,State,County,City,Neighborhood,RegionID,2016_evict_rate,2016_evict_filing_rate,2011-2015_rent_perc_change,2015-2019_rent_perc_change,potential_bad_landlord,bad_landlord_address,num_stops_quart_mi,num_stops_half_mi,num_stops_3quart_mi,num_stops_1_mi
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4274104,"6637 S Greenwood Ave , Chicago, IL 60637",900,Apt,1.0,2.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.773893,-87.599352,POINT (-87.599352 41.773893),170318344003,IL,Cook,Chicago,Woodlawn,276138,2.69,4.93,-0.027008,0.024105,False,NaN,0,0,2,4
4276320,"6526 S Drexel Ave , Chicago, IL 60637",1000,Apt,1.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.776000,-87.603700,POINT (-87.6037 41.776),170314208002,IL,Cook,Chicago,Woodlawn,276138,3.46,8.07,-0.027008,0.024105,False,NaN,0,2,4,4
4284069,"6530 S Greenwood Ave 2, Chicago, IL 60637",950,Apt,1.0,2.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.775929,-87.599753,POINT (-87.59975319999999 41.7759289),170318344003,IL,Cook,Chicago,Woodlawn,276138,2.69,4.93,-0.027008,0.024105,False,NaN,0,2,2,4
4296404,"6319 S Ellis Ave 2, Chicago, IL 60637",1000,Apt,1.0,3.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.779811,-87.600427,POINT (-87.6004269 41.7798106),170318344001,IL,Cook,Chicago,Woodlawn,276138,0.69,1.74,-0.027008,0.024105,False,NaN,0,2,2,4
4552033,"6437 S Drexel Ave 2b, Chicago, IL 60637",977,Apt,1.0,2.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.777500,-87.602800,POINT (-87.6028 41.7775),170314208001,IL,Cook,Chicago,Woodlawn,276138,1.72,4.60,-0.027008,0.024105,False,NaN,2,2,4,4
4574061,"925 E Marquette Rd , Chicago, IL 60637",1000,Apt,1.0,2.0,Yes,Available Now,(773) 285-3310,http://chicagoha.gosection8.com/Section-8-hous...,41.775000,-87.602300,POINT (-87.6023 41.775),170314208002,IL,Cook,Chicago,Woodlawn,276138,3.46,8.07,-0.027008,0.024105,False,NaN,0,2,2,4
4619162,"6200 S University Ave , Chicago, IL 60637",911,Apt,1.5,3.0,Yes,Check Availability,NaN,http://chicagoha.gosection8.com/Section-8-hous...,41.782100,-87.598300,POINT (-87.59829999999999 41.7821),170314203001,IL,Cook,Chicago,Woodlawn,276138,2.28,6.83,-0.027008,0.024105,False,NaN,0,2,2,4
4641459,"6619 S Greenwood Ave , Chicago, IL 60637",1000,Apt,1.0,2.0,Yes,Available Now,(312) 852-3387,http://chicagoha.gosection8.com/Section-8-hous...,41.774300,-87.599700,POINT (-87.5997 41.7743),170318344003,IL,Cook,Chicago,Woodlawn,276138,2.69,4.93,-0.027008,0.024105,False,NaN,0,0,2,4
4647666,"6619 S Greenwood Ave , Chicago, IL 60637",980,Apt,1.0,2.0,Yes,Available Now,(312) 852-3387,http://chicagoha.gosection8.com/Section-8-hous...,41.774300,-87.599700,POINT (-87.5997 41.7743),170318344003,IL,Cook,Chicago,Woodlawn,276138,2.69,4.93,-0.027008,0.024105,False,NaN,0,0,2,4


In [328]:
user.search(cr, db, user.OUTPUT1)

1 search results saved in output/sample_output1.csv


In [284]:
z = pd.read_csv("data/Neighborhood_Zri_AllHomesPlusMultifamily.csv", usecols=["City", "RegionName"])

In [281]:
zz = pd.read_csv( "processed_data/zillow_rindex_with_increase.csv")

In [285]:
z

,RegionName,City
0,Northeast Dallas,Dallas
1,Maryvale,Phoenix
2,Upper West Side,New York
3,South Los Angeles,Los Angeles
4,Upper East Side,New York
5,Sunrise Manor,Las Vegas
6,Southeast Los Angeles,Los Angeles
7,East New York,New York
8,Paradise Valley,Phoenix
9,Washington Heights,New York


In [297]:
z[z.City == 'Chicago'].groupby('RegionName').groups

{'Albany Park': Int64Index([142], dtype='int64'),
 'Andersonville': Int64Index([4085], dtype='int64'),
 'Arcadia Terrace': Int64Index([2698], dtype='int64'),
 'Archer Heights': Int64Index([150], dtype='int64'),
 'Ashburn': Int64Index([904], dtype='int64'),
 'Avalon Park': Int64Index([3801], dtype='int64'),
 'Avondale': Int64Index([687], dtype='int64'),
 'Back of the Yards': Int64Index([289], dtype='int64'),
 'Belmont Central': Int64Index([357], dtype='int64'),
 'Belmont Gardens': Int64Index([1037], dtype='int64'),
 'Belmont Heights': Int64Index([1023], dtype='int64'),
 'Belmont Terrace': Int64Index([4316], dtype='int64'),
 'Beverly': Int64Index([924], dtype='int64'),
 'Beverly Woods': Int64Index([5794], dtype='int64'),
 'Big Oaks': Int64Index([2911], dtype='int64'),
 'Bowmanville': Int64Index([3348], dtype='int64'),
 'Brainerd': Int64Index([938], dtype='int64'),
 'Bridgeport': Int64Index([241], dtype='int64'),
 'Brighton Park': Int64Index([188], dtype='int64'),
 'Bronzeville': Int64Ind

In [278]:
l

['Albany Park',
 'Andersonville',
 'Arcadia Terrace',
 'Archer Heights',
 'Ashburn',
 'Avalon Park',
 'Avondale',
 'Back of the Yards',
 'Belmont Central',
 'Belmont Gardens',
 'Belmont Heights',
 'Belmont Terrace',
 'Beverly',
 'Beverly Woods',
 'Big Oaks',
 'Bowmanville',
 'Brainerd',
 'Bridgeport',
 'Brighton Park',
 'Bronzeville',
 'Bucktown',
 'Budlong Woods',
 'Burnside',
 'Cabrini Green',
 'Calumet Heights',
 'Canaryville',
 'Chatham',
 'Chicago Lawn',
 'Chinatown',
 'Chrysler Village',
 'Clearing',
 'Cottage Grove Heights',
 'Cragin',
 'Dearborn Park',
 'Dunning',
 'East Beverly',
 'East Chatham',
 'East Garfield Park',
 'East Hyde Park',
 'East Side',
 'East Ukrainian Village',
 'Edgebrook',
 'Edgewater',
 'Edgewater Glen',
 'Edison Park',
 'Englewood',
 'Fernwood',
 'Ford City',
 'Forest Glen',
 'Fulton River District',
 'Gage Park',
 'Galewood',
 'Garfield Ridge',
 'Gold Coast',
 'Goose Island',
 'Graceland West',
 'Grand Crossing',
 'Greektown',
 'Gresham',
 'Groveland Park